# Amazon SageMaker scikit-learn Bring Your Own Model
_**Hosting a pre-trained scikit-learn Model in Amazon SageMaker scikit-learn Container**_

---

---

## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

This notebook shows how to use a pre-trained scikit-learn model with the Amazon SageMaker scikit-learn container to quickly create a hosted endpoint for that model.
We use the California Housing dataset, present in Scikit-Learn: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html. The California Housing dataset was originally published in:

> Pace, R. Kelley, and Ronald Barry. "Sparse spatial auto-regressions." Statistics & Probability Letters 33.3 (1997): 291-297.

---
## Setup

Let's start by specifying:

* AWS region.
* The IAM role arn used to give learning and hosting access to your data.
* The S3 bucket that you want to use for training and model data.

In [ ]:
import os
import boto3
import re
import json
import time
import datetime
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.model import SKLearnModel
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-sklearn-byo-model"

print(f"bucket: {bucket}")

## Prepare data for model inference

We load the California housing dataset from sklearn, and will use it to invoke SageMaker Endpoint

In [ ]:
data = fetch_california_housing()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

# we don't train a model, so we will need only the testing data
testX = pd.DataFrame(X_test, columns=data.feature_names)

In [ ]:
testX.head(10)

## Download a pre-trained model file

Download a pretrained Scikit-Learn Random Forest model.

We used the California Housing dataset, present in Scikit-Learn: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset to train the model.

For more details on how to train the model with Amazon SageMaker, please refer to the [Develop, Train, Optimize and Deploy Scikit-Learn Random Forest notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_randomforest/Sklearn_on_SageMaker_end2end.ipynb)

In [ ]:
!aws s3 cp s3://aws-ml-blog/artifacts/scikit_learn_bring_your_own_model/model.joblib .

### Compressed the model file to a GZIP tar archive 

Note that the model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`. The model file needs to be tar-zipped. 

In [ ]:
model_file_name = "model.joblib"

In [ ]:
!tar czvf model.tar.gz $model_file_name

## Upload the pre-trained model `model.tar.gz` file to S3

In [ ]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

## Set up hosting for the model

This involves creating a SageMaker model from the model file previously uploaded to S3.

In [ ]:
model_data = "s3://{}/{}".format(bucket, key)
print(f"model data: {model_data}")

### Write the Inference Script

When using endpoints with the Amazon SageMaker managed `Scikit Learn` container, we need to provide an entry point script for inference that will **at least** load the saved model.

After the SageMaker model server has loaded your model by calling `model_fn`, SageMaker will serve your model. Model serving is the process of responding to inference requests, received by SageMaker `InvokeEndpoint` API calls.


We will implement also the `predict_fn()` function that takes the deserialized request object and performs inference against the loaded model.

We will now create this script and call it `inference.py` and store it at the root of a directory called `code`.

**Note:** You would modify the script below to implement your own inferencing logic.

Additional information on model loading and model serving for scikit-learn on SageMaker can be found in the [SageMaker Scikit-learn Model Server documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#deploy-a-scikit-learn-model)

There are also several functions for hosting which we won't define,
 - `input_fn()` - Takes request data and deserializes the data into an object for prediction.
 - `output_fn()` - Takes the result of prediction and serializes this according to the response content type.

These will take on their default values as described [SageMaker Scikit-learn Serve a Model documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#serve-a-model)

In [ ]:
!pygmentize ./code/inference.py

### Installing additional Python dependencies

It also may be necessary to supply a `requirements.txt` file to ensure any necessary dependencies are installed in the container along with the script. For this script, in addition to the Python standard libraries, we showcase how to install the `boto3` `requests`, and `nltk` libraries.

In [ ]:
!pygmentize ./code/requirements.txt

### Deploy with Python SDK

Here we showcase the process of creating a model from s3 artifacts, that could be used to deploy a model that was trained in a different session or even out of SageMaker.

In [ ]:
model = SKLearnModel(
    role=role,
    model_data=model_data,
    framework_version="0.23-1",
    py_version="py3",
    source_dir="code",
    entry_point="inference.py",
)

### Create endpoint
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 5-10 minutes to complete.

In [ ]:
%%time

predictor = model.deploy(instance_type="ml.t2.medium", initial_instance_count=1)

## Validate the model for use
Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint.

### Invoke with the Python SDK

Let's generate the prediction for a single data point. We'll pick one from the test data generated earlier.

In [ ]:
# the SKLearnPredictor does the serialization from pandas for us
predictions = predictor.predict(testX[data.feature_names])
print(predictions)

### Alternative: invoke with `boto3`

This is useful when invoking the model from external clients, e.g. Lambda Functions, or other micro-services.

In [ ]:
runtime = boto3.client("sagemaker-runtime")

#### Option 1: `csv` serialization

In [ ]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

print(response["Body"].read())

#### Option 2: `npy` serialization

In [ ]:
# npy serialization
from io import BytesIO


# Serialise numpy ndarray as bytes
buffer = BytesIO()
# Assuming testX is a data frame
np.save(buffer, testX[data.feature_names].values)

response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint, Body=buffer.getvalue(), ContentType="application/x-npy"
)

print(response["Body"].read())

### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
predictor.delete_endpoint()

## Optional - Deploy a model from `SageMaker Model Registry` and update an existing SageMaker Endpoint

## Upload `sourcedir.tar.gz` to S3

When using the SageMaker Python SDK, it will upload the files in `entry_point`, `source_dir`, and dependencies to S3 as a compressed `sourcedir.tar.gz` file 

Because you are deploying a model you trained outside of SageMaker with `boto3`, this is something you will have to do.

In [ ]:
!cd code && tar czvf ../sourcedir.tar.gz *

In [ ]:
fObj = open("sourcedir.tar.gz", "rb")
key = os.path.join(prefix, "sourcedir.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

In [ ]:
sourcedir_data = "s3://{}/{}".format(bucket, key)
print(f"sources data: {sourcedir_data}")

Let's inspect the content of this bucket

In [ ]:
!aws s3 ls s3://{bucket}/{prefix}/

## Create a Model Group

This will create a model group. A model group contains a group of model versions.

In [ ]:
client = boto3.client("sagemaker")

model_package_group_name = "sklearn-california-housing-" + str(round(time.time()))
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "My sample sklearn model package group"
}

create_model_pacakge_group_response = client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))

## Register a Model Version for the 1st model

You will create a model package that you will use to create a SageMaker versioned model that is part of a model group.

Nw create create a model package by specifying a Docker container that contains your inference code and the Amazon S3 location of your model artifacts, provide values for `InferenceSpecification`.

In [ ]:
training_image = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3"
sagemaker_program = "inference.py"

modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": training_image,
             "Environment": {
                "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
                "SAGEMAKER_PROGRAM": sagemaker_program,
                "SAGEMAKER_REGION": region,
                "SAGEMAKER_SUBMIT_DIRECTORY": sourcedir_data
             },
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_data

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to predict California housing values",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

In [ ]:
create_mode_package_response = client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

## View Model Groups and Versions

In [ ]:
list_model_packages_response = client.list_model_packages(ModelPackageGroupName=model_package_group_name)
list_model_packages_response

You should now see model package version 1 in the `model package` ARN.

In [ ]:
model_version_arn = list_model_packages_response['ModelPackageSummaryList'][0]['ModelPackageArn']
print(model_version_arn)

## View 1st Model Version Details

In [ ]:
client.describe_model_package(ModelPackageName=model_version_arn)

## Update 1st Model Approval Status

In [ ]:
model_package_update_input_dict = {
    "ModelPackageArn" : model_package_arn,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = client.update_model_package(**model_package_update_input_dict)
model_package_update_response

## Deploy the 1st Model in the Registry

In [ ]:
model_1_name = 'DEMO-sklearn-califonia-housing-model-1-' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print("Model name : {}".format(model_1_name))

In [ ]:
primary_container = {
    'ModelPackageName': model_version_arn,
}

## Create `Model` 

In [ ]:
create_model_respose = client.create_model(
    ModelName = model_1_name,
    ExecutionRoleArn = get_execution_role(),
    PrimaryContainer = primary_container
)

print("Model arn : {}".format(create_model_respose["ModelArn"]))

## Create an Endpoint Config from 1st model

This will create an endpoint configuration that Amazon SageMaker hosting services uses to deploy models. In the configuration, you identify one or more models, created using the `CreateModel` API, to deploy and the resources that you want Amazon SageMaker to provision. Then you call the `CreateEndpoint` API.

More info on `create_endpoint_config` can be found on the [Boto3 SageMaker documentation page](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config).

In [ ]:
endpoint_config_1_name = "sklearn-endpoint-config-1-" + datetime.datetime.now().strftime(
    "%Y-%m-%d-%H-%M-%S"
)

endpoint_config_1_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_1_name,
    ProductionVariants=[
        {
            "VariantName": "AllTrafficVariant",
            "ModelName": model_1_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.c5.large",
            "InitialVariantWeight": 1,
        },
    ],
)

endpoint_config_1_response

## Deploy the 1st Endpoint Config to a real-time endpoint

This will create an endpoint using the endpoint configuration specified in the request. Amazon SageMaker uses the endpoint to provision resources and deploy models. Note that you have already created the endpoint configuration with the `CreateEndpointConfig` API in the previous step.

More info on `create_endpoint` can be found on the [Boto3 SageMaker documentation page](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint).

In [ ]:
endpoint_name = "sklearn-endpoint-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_1_name,
)

create_endpoint_response

## Wait for Endpoint to be ready

In [ ]:
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

## Invoke Endpoint with `boto3`

In [ ]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

print(response["Body"].read())

## Register a Model Version for the 2nd model

You will now register a Model Version for the 2nd model in `Model Registry`. 

In real life, you will use a new trained model artifacts. In this notebook, you will use the same artifcats of the 1st model.

In [ ]:
modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": training_image,
             "Environment": {
                "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
                "SAGEMAKER_PROGRAM": sagemaker_program,
                "SAGEMAKER_REGION": region,
                "SAGEMAKER_SUBMIT_DIRECTORY": sourcedir_data
             },
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_data

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to predict California housing values",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

You should now see model package version 2 in the `model package` ARN.

In [ ]:
create_mode_package_response = client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

## View Model Groups and Versions

You should see there are two models registered - `/1` and `/2`.

In [ ]:
list_model_packages_response = client.list_model_packages(ModelPackageGroupName=model_package_group_name)
list_model_packages_response

In [ ]:
model_version_arn = list_model_packages_response['ModelPackageSummaryList'][0]['ModelPackageArn']
print(model_version_arn)

## View 2nd Model Version Details

In [ ]:
client.describe_model_package(ModelPackageName=model_version_arn)

## Update the 2nd Model Approval Status

In [ ]:
model_package_update_input_dict = {
    "ModelPackageArn" : model_package_arn,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = client.update_model_package(**model_package_update_input_dict)
model_package_update_response

## Deploy the 2nd Model in the Registry

In [ ]:
model_2_name = 'DEMO-sklearn-califonia-housing-model-2-' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print("Model name : {}".format(model_2_name))

In [ ]:
primary_container = {
    'ModelPackageName': model_version_arn,
}

In [ ]:
create_model_respose = client.create_model(
    ModelName = model_2_name,
    ExecutionRoleArn = get_execution_role(),
    PrimaryContainer = primary_container
)

print("Model arn : {}".format(create_model_respose["ModelArn"]))

In [ ]:
endpoint_config_2_name = "sklearn-endpoint-config-2-" + datetime.datetime.now().strftime(
    "%Y-%m-%d-%H-%M-%S"
)

endpoint_config_2_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_2_name,
    ProductionVariants=[
        {
            "VariantName": "AllTrafficVariant",
            "ModelName": model_2_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.c5.large",
            "InitialVariantWeight": 1,
        },
    ],
)

endpoint_config_2_response

## Update the real-time endpoint with the 2nd Endpoint Config

This will deploy the new `EndpointConfig` specified in the request, switches to using newly created endpoint, and then deletes resources provisioned for the endpoint using the previous `EndpointConfig` (there is no availability loss).

More info on `update_endpoint` can be found on the [Boto3 SageMaker documentation page](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.update_endpoint).

In [ ]:
update_endpoint_response = client.update_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_2_name
)

update_endpoint_response

## Wait for Endpoint to be ready

Navigating to the SageMaker Endpoints, in `SageMaker Components and registries` tab, you'll see the endpoint in `Updating` status.

In [ ]:
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Updating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

## Invoke Endpoint with `boto3`

In [ ]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

print(response["Body"].read())

## View `Model Registry` in SageMaker Studio

In SageMaker Studio, choose `SageMaker Components and registries` in the left pane, and under `Model Registry` choose the `sklearn-california-housing` model group. 

You should see two versions of the model.

![](img/sm_studio_model_registry.png "View Model Registry in SageMaker Studio")

## Clean up

Endpoints should be deleted when no longer in use, since (per the [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/)) they're billed by time deployed.

In [ ]:
client.delete_endpoint(EndpointName=endpoint_name)

## Optional - Deploy a Model Version from a Different Account

Enable an AWS account to deploy model versions that were created in a different account by adding a cross-account resource policy. For example, one team in your organization might be responsible for training models, and a different team is responsible for deploying and updating models. When you create resource policies, you apply the policy to the resource to which you want to grant access. For more information about cross-account resource policies in AWS, see [Cross-account policy evaluation logic](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_evaluation-logic-cross-account.html) in the AWS Identity and Access Management User Guide.

To enable cross-account model deployment in SageMaker, you have to provide a cross-account resource policy for the model group that contains the model versions you want to deploy, the Amazon ECR repository where the inference image for the model group resides, and the Amazon S3 bucket where the model versions are stored. 

The following example creates cross-account policies for group that contains the model versions, and the Amazon S3 bucket where the model versions are stored, and applies the policies to the resources.

Let's define the AWS cross account id to grant access to.

In [ ]:
account = role.split(':')[4]

cross_account_id = "<YOUR CROSS AWS ACCOUNT TO GRANT ACCESS TO>"

print(f'Account id: {account}')
print(f'Cross Account id: {cross_account_id}')

## Create policy for access to the S3 bucket

In [ ]:
bucket_policy = {
    'Version': '2012-10-17',
    'Statement': [{
        'Sid': 'AddPerm',
        'Effect': 'Allow',
        'Principal': {
            'AWS': f'arn:aws:iam::{cross_account_id}:root'
        },
        'Action': 's3:*',
        'Resource': f'arn:aws:s3:::{bucket}/*'
    }]
}

bucket_policy

## Convert the policy from JSON dict to string

In [ ]:
bucket_policy = json.dumps(bucket_policy)

## Set the new S3 policy

In [ ]:
s3 = boto3.client('s3')
respose = s3.put_bucket_policy(
    Bucket = bucket,
    Policy = bucket_policy)

## Create policy for access to the ModelPackageGroup

In [ ]:
model_pacakge_group_policy = {
    'Version': '2012-10-17',
    'Statement': [{
        'Sid': 'AddPermModelPackageGroup',
        'Effect': 'Allow',
        'Principal': {
            'AWS': f'arn:aws:iam::{cross_account_id}:root'
        },
        'Action': ['sagemaker:DescribeModelPackageGroup'],
        'Resource': f'arn:aws:sagemaker:{region}:{account}:model-package-group/{model_package_group_name}'
    },{
        'Sid': 'AddPermModelPackageVersion',
        'Effect': 'Allow',
        'Principal': {
            'AWS': f'arn:aws:iam::{cross_account_id}:root'
        },
        'Action': ["sagemaker:DescribeModelPackage",
                   "sagemaker:ListModelPackages",
                   "sagemaker:UpdateModelPackage",
                   "sagemaker:CreateModel"],
        'Resource': f'arn:aws:sagemaker:{region}:{account}:model-package/{model_package_group_name}/*'
    }]
}

model_pacakge_group_policy

## Convert the policy from JSON dict to string

In [ ]:
model_pacakge_group_policy = json.dumps(model_pacakge_group_policy)

## Set the new policy

In [ ]:
respose = client.put_model_package_group_policy(
    ModelPackageGroupName = model_package_group_name,
    ResourcePolicy = model_pacakge_group_policy)

In [ ]:
print('ModelPackageGroupArn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))